# Implementação e Exemplos

## Função Custo (Erro)

### EQM - Erro Quadrático Médio

$$
EQM = \frac{1}{n} \sum_{i} (y_{esperado} - y_{estimado})^2
$$
$$
\frac{\partial EQM}{\partial Y} = \frac{2}{n} (y_{estimado} - y_{esperado})
$$

In [32]:
import numpy as np

def erro_quadratico_medio(y_esperado: np.ndarray, y_calculado: np.ndarray) -> np.ndarray:
    return np.mean(np.power(y_esperado-y_calculado, 2))

def derivada_erro_quadratico_medio(y_esperado: np.ndarray, y_calculado: np.ndarray) -> np.ndarray:
    n = y_esperado.size
    return 2*(y_calculado - y_esperado)/n

### Outras opções

https://www.tensorflow.org/api_docs/python/tf/keras/losses

## Uso em uma Rede Neural

https://towardsdatascience.com/math-neural-network-from-scratch-in-python-d6da9f29ce65

### Rede Neural - Camadas

In [33]:
from abc import ABC
import numpy as np
from utils import array_to_latex


class Camada(ABC):
    def __init__(self):
        self.entrada: np.ndarray = None
        self.saida: np.ndarray = None

    def forward_propagation(self, input, logs: list):
        raise NotImplementedError

    def backward_propagation(
        self, output_error, taxa_aprendizado: np.number, logs: list
    ):
        raise NotImplementedError


class FCLayer(Camada):
    def __init__(self, tamanho_entrada: int, tamanho_saida: int):
        self.pesos = np.random.rand(tamanho_entrada, tamanho_saida)
        self.bias = np.random.rand(1, tamanho_saida)

    def forward_propagation(self, entrada: np.ndarray, logs: list) -> np.ndarray:
        self.entrada = entrada
        self.saida = np.dot(self.entrada, self.pesos) + self.bias

        logs.append(r"Y = Xw + b")
        logs.append(
            r"\begin{gather}"
            + array_to_latex(self.saida, "")
            + r" = "
            + array_to_latex(self.entrada, "")
            + r" \cdot "
            + array_to_latex(self.pesos, "")
            + r" + "
            + array_to_latex(self.bias, "")
            + r"\end{gather}"
        )

        return self.saida

    # computes dE/dW, dE/dB for a given output_error=dE/dY. Returns input_error=dE/dX.
    def backward_propagation(
        self, derivada_erro: np.ndarray, taxa_aprendizado: np.number, logs: list
    ) -> np.ndarray:
        derivada_erro_entrada = np.dot(derivada_erro, self.pesos.T)

        logs.append(r"\nabla f_{custo} = \partial E \cdot \partial W")
        logs.append(
            array_to_latex(derivada_erro_entrada, "")
            + r" = "
            + array_to_latex(derivada_erro, "")
            + r" \cdot "
            + array_to_latex(self.pesos.T, "")
        )

        pesos_erro = np.dot(self.entrada.T, derivada_erro)
        pesos_atualizados = self.pesos - taxa_aprendizado * pesos_erro
        bias_atualizados = self.bias - taxa_aprendizado * derivada_erro

        logs.append(r"W = W - \lambda (X \cdot \nabla f_{custo})")
        logs.append(
            r"\begin{gather}"
            + array_to_latex(pesos_atualizados, "")
            + r" = "
            + array_to_latex(self.pesos, "")
            + f" - {taxa_aprendizado}"
            + array_to_latex(self.entrada.T, "")
            + r" \cdot "
            + array_to_latex(derivada_erro, "")
            + r"\end{gather}"
        )

        self.pesos = pesos_atualizados
        self.bias = bias_atualizados

        return derivada_erro_entrada


class ActivationLayer(Camada):
    def __init__(self, fn_de_ativacao, derivada_fn_de_ativacao):
        self.fn_de_ativacao = fn_de_ativacao
        self.derivada_fn_de_ativacao = derivada_fn_de_ativacao

    def forward_propagation(self, entrada: np.ndarray, logs: list):
        self.entrada = entrada
        self.saida = self.fn_de_ativacao(self.entrada)

        logs.append(r"Y = \sigma(X)")
        logs.append(
            r"\begin{gather}"
            + array_to_latex(self.saida, "")
            + r" = \sigma("
            + array_to_latex(self.entrada, "")
            + r") "
            + r"\end{gather}"
        )

        return self.saida

    # Returns input_error=dE/dX for a given output_error=dE/dY.
    # regra da cadeia
    def backward_propagation(
        self, derivada_erro: np.ndarray, taxa_aprendizado: np.number, logs: list
    ):
        derivada_erro_entrada = derivada_erro * self.derivada_fn_de_ativacao(
            self.entrada
        )
        logs.append(
            r"\nabla = \frac{\partial E}{\partial Y} \frac{\partial Y}{\partial X}"
        )
        logs.append(
            r"\begin{gather}"
            + array_to_latex(derivada_erro_entrada, "")
            + r" = "
            + array_to_latex(derivada_erro, "")
            + r" \cdot "
            + r" \sigma^{'}("
            + array_to_latex(self.entrada, "")
            + r") \end{gather}"
        )

        return derivada_erro_entrada


### Rede Neural - Rede

In [34]:
class RedeNeural:
    def __init__(self, fn_custo: callable, derivada_fn_custo: callable):
        self.camadas = []
        self.logs = []
        self.fn_custo = fn_custo
        self.derivada_fn_custo = derivada_fn_custo

    def add(self, camada):
        self.camadas.append(camada)

    def predict(self, entrada: np.ndarray) -> np.ndarray:
        n_entradas = len(entrada)
        resultado = []

        for n in range(n_entradas):
            saida = entrada[n]
            for camada in self.camadas:
                saida = camada.forward_propagation(saida, [])
            resultado.append(saida)

        return resultado

    def treinar(
        self,
        vetor_x: np.ndarray,
        y_esperado: np.ndarray,
        max_n_iteracoes: int,
        taxa_aprendizado: np.number,
    ):
        n_amostras = len(vetor_x)
        for iteracao in range(max_n_iteracoes):
            self.logs.append([])
            soma_erros = 0
            for n in range(n_amostras):

                output = vetor_x[n]
                for indice_camada, camada in enumerate(self.camadas):
                    self.logs[iteracao].append(f"Camada: {indice_camada}")
                    output = camada.forward_propagation(output, self.logs[iteracao])

                erro = self.fn_custo(y_esperado[n], output)
                soma_erros += erro
                self.logs[iteracao].append(
                    r"f_{custo}("
                    + array_to_latex(y_esperado[n], "")
                    + r","
                    + array_to_latex(output, "")
                    + r") = "
                    + array_to_latex(erro, "")
                )

                valor_derivada_erro = self.derivada_fn_custo(y_esperado[n], output)
                self.logs[iteracao].append(
                    r"\nabla = \frac{\partial E}{\partial Y} = "
                    + array_to_latex(valor_derivada_erro, "")
                )
                for indice_camada, camada in enumerate(reversed(self.camadas)):
                    self.logs[iteracao].append(
                        f"Camada: {len(self.camadas) - indice_camada -1}"
                    )
                    valor_derivada_erro = camada.backward_propagation(
                        valor_derivada_erro, taxa_aprendizado, self.logs[iteracao]
                    )

            media_erros = soma_erros / n_amostras
            if iteracao < 5 or np.log10(iteracao + 1).is_integer():
                print(
                    "Iteração %d/%d   MédiaErro=%f"
                    % (iteracao + 1, max_n_iteracoes, media_erros)
                )
            if media_erros < 0.01:
                print(
                    "Iteração %d/%d MédiaErro=%f"
                    % (iteracao + 1, max_n_iteracoes, media_erros)
                )
                break


### Rede Neural - Função de Ativação (Sigmóide)

$$
\sigma(x) = \frac{1}{1 - e^{-x}} 
$$

$$
\frac{d \sigma}{dx} = \sigma(x) (1 - \sigma(x))
$$

https://towardsdatascience.com/derivative-of-the-sigmoid-function-536880cf918e

In [35]:
import numpy as np


def sigmoide(x: np.ndarray) -> np.ndarray:
    return 1 / (1 + np.exp(-x))


def derivada_sigmoide(x: np.number) -> np.number:
    sigmoide_x = sigmoide(x)
    return sigmoide_x * (1 - sigmoide_x)


## Exemplo: Ou Exclusivo (XOR)

Operador retorna verdadeiro apenas caso entradas A ou B {(verdadeiro, falso) ou (falso, verdadeiro)} sejam verdadeiras, retornando falso caso ambas sejam verdadeiras ou ainda caso nenhuma seja.

### Entradas e Saídas Esperadas

In [36]:
vetor_x_exemplo_XOR = np.array([[[0, 0]], [[0, 1]], [[1, 0]], [[1, 1]]])
vetor_y_esperado_exemplo_XOR = np.array([[[0]], [[1]], [[1]], [[0]]])


### Rede



In [37]:
rede_exemplo_XOR = RedeNeural(erro_quadratico_medio, derivada_erro_quadratico_medio)
rede_exemplo_XOR.add(FCLayer(2, 2))
rede_exemplo_XOR.add(ActivationLayer(sigmoide, derivada_sigmoide))
rede_exemplo_XOR.add(FCLayer(2, 1))


### Treinamento

In [38]:
from IPython.display import Latex

rede_exemplo_XOR.treinar(
    vetor_x_exemplo_XOR,
    vetor_y_esperado_exemplo_XOR,
    max_n_iteracoes=1000,
    taxa_aprendizado=0.5,
)


for iteracao_log in rede_exemplo_XOR.logs[0:1]:
    for log in iteracao_log:
        display(Latex(log))

display(Latex(r'Última \quad iteração:'))

for log in rede_exemplo_XOR.logs[-1]:
    display(Latex(log))

Iteração 1/1000   MédiaErro=4.351091
Iteração 2/1000   MédiaErro=0.697102
Iteração 3/1000   MédiaErro=0.492345
Iteração 4/1000   MédiaErro=0.492510
Iteração 5/1000   MédiaErro=0.494672
Iteração 10/1000   MédiaErro=0.549432
Iteração 65/1000 MédiaErro=0.009517


<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

...

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

### Validação

In [39]:
from utils import array_to_latex
from IPython.display import Latex

validacao_exemplo_XOR = rede_exemplo_XOR.predict(vetor_x_exemplo_XOR)
validacao_exemplo_XOR = np.asarray(validacao_exemplo_XOR) \
                          .reshape(4, 1)

Latex(r'\begin{gather}' +
      array_to_latex(vetor_x_exemplo_XOR.reshape(4,2), 'X=') +
      r'\rightarrow' +
      array_to_latex(validacao_exemplo_XOR, '') +
      r'\end{gather}')

<IPython.core.display.Latex object>

## Exemplo: Display de 7 Segmentos

### Entradas e Saídas Esperadas

In [40]:
import pandas as pd
from IPython.display import display, Latex

segmentos = ['A', 'B', 'C', 'D', 'E', 'F', 'G']

tabela_segmentos_digitos = pd.DataFrame([
    #A, B, C, D, E, F, G
    [1, 1, 1, 1, 1, 1, 0],
    [0, 1, 1, 0, 0, 0, 0],
    [1, 1, 0, 1, 1, 0, 1],
    [1, 1, 1, 1, 0, 0, 1],
    [0, 1, 1, 0, 0, 1, 1],
    [1, 0, 1, 1, 0, 1, 1],
    [0, 0, 1, 1, 1, 1, 1],
    [1, 1, 1, 0, 0, 0, 0],
    [1, 1, 1, 1, 1, 1, 1],
    [1, 1, 0, 0, 1, 1, 1],
], columns=segmentos)

display(tabela_segmentos_digitos)

vetor_y_esperado_exemplo_segmentos = np.array(
    [
        [[1, 0, 0, 0, 0, 0, 0, 0, 0, 0]],
        [[0, 1, 0, 0, 0, 0, 0, 0, 0, 0]],
        [[0, 0, 1, 0, 0, 0, 0, 0, 0, 0]],
        [[0, 0, 0, 1, 0, 0, 0, 0, 0, 0]],
        [[0, 0, 0, 0, 1, 0, 0, 0, 0, 0]],
        [[0, 0, 0, 0, 0, 1, 0, 0, 0, 0]],
        [[0, 0, 0, 0, 0, 0, 1, 0, 0, 0]],
        [[0, 0, 0, 0, 0, 0, 0, 1, 0, 0]],
        [[0, 0, 0, 0, 0, 0, 0, 0, 1, 0]],
        [[0, 0, 0, 0, 0, 0, 0, 0, 0, 1]],
    ]
)

Latex(r"\begin{gather}" +
      array_to_latex(vetor_y_esperado_exemplo_segmentos.reshape(10, 10), r"Y_{esperado}=") +
      r"\end{gather}")

,A,B,C,D,E,F,G
0,1,1,1,1,1,1,0
1,0,1,1,0,0,0,0
2,1,1,0,1,1,0,1
3,1,1,1,1,0,0,1
4,0,1,1,0,0,1,1
5,1,0,1,1,0,1,1
6,0,0,1,1,1,1,1
7,1,1,1,0,0,0,0
8,1,1,1,1,1,1,1
9,1,1,0,0,1,1,1


<IPython.core.display.Latex object>

### Rede

In [41]:
vetor_x_exemplo_segmentos = tabela_segmentos_digitos.values.reshape((10, 1, 7))

rede_exemplo_display = RedeNeural(erro_quadratico_medio, derivada_erro_quadratico_medio)
rede_exemplo_display.add(FCLayer(7, 14))
rede_exemplo_display.add(ActivationLayer(sigmoide, derivada_sigmoide))
rede_exemplo_display.add(FCLayer(14, 10))


### Treinamento

In [42]:
rede_exemplo_display.treinar(
    vetor_x_exemplo_segmentos,
    vetor_y_esperado_exemplo_segmentos,
    max_n_iteracoes=1000,
    taxa_aprendizado=0.7,
)

for log in rede_exemplo_display.logs[0]:
    display(Latex(log))


Iteração 1/1000   MédiaErro=7.651644
Iteração 2/1000   MédiaErro=0.161469
Iteração 3/1000   MédiaErro=0.148037
Iteração 4/1000   MédiaErro=0.137577
Iteração 5/1000   MédiaErro=0.129097
Iteração 10/1000   MédiaErro=0.102132
Iteração 100/1000   MédiaErro=0.037707
Iteração 279/1000 MédiaErro=0.009898


<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

...

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

### Validação

In [43]:
from utils import array_to_latex
from IPython.display import Latex

validacao_exemplo_display = rede_exemplo_display.predict(vetor_x_exemplo_segmentos)

validacao_exemplo_display = np.asarray(validacao_exemplo_display) \
                              .reshape(10, 10)                    \
                              .round(decimals = 2)

Latex(r'\begin{gather}' +
      array_to_latex(vetor_x_exemplo_segmentos.reshape(10,7), 'X=') +
      r'\rightarrow' +
      array_to_latex(validacao_exemplo_display, '') +
      r'\end{gather}')

<IPython.core.display.Latex object>